In [1]:
import pandas as pd
train_df = pd.read_csv('train_processed.csv')

In [10]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report
from joblib import Parallel, delayed
from sklearn.decomposition import TruncatedSVD

In [5]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

def tokenize_text(text):
    return word_tokenize(str(text))

train_df['Text_Processed_Tokens'] = train_df['Text_Processed'].apply(tokenize_text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [6]:
# Used to pass into N-gram and Vector Space
X = train_df['Text_Processed'].tolist()
y = [score - 1 for score in train_df['Score'].tolist()]

# Used to pass into word embeddings
X_token = train_df['Text_Processed_Tokens']

# Splitting the dataset to 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "Naïve Bayes": MultinomialNB(),
    "Gradient Boosting": GradientBoostingClassifier()
}
results = []

In [11]:
ngram_models = {
    "Unigram": CountVectorizer(ngram_range=(1, 1)),
    "Bigram": CountVectorizer(ngram_range=(2, 2)),
    "Trigram": CountVectorizer(ngram_range=(3, 3)),
    "Combined N-Grams": CountVectorizer(ngram_range=(1, 3))
}

In [12]:
# Iterate over classifiers and n-gram models
for clf_name, clf in classifiers.items():
    print("=======================================================")
    print(clf_name)
    print("=======================================================")
    for ngram_name, vectorizer in ngram_models.items():
        pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', clf)
        ])

        # Train and test the model
        pipeline.fit(X_train, y_train)
        y_pred = cross_val_predict(pipeline, X_test, y_test, cv=5)

        # Evaluate performance
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        print ("---------------------------------------------")
        print(ngram_name)
        print ("---------------------------------------------")
        print (classification_report(y_test, y_pred, zero_division=1))

        # Append results to list
        results.append(["N-Gram", clf_name, ngram_name, accuracy, precision, recall, f1])

Logistic Regression


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------------------------
Unigram
---------------------------------------------
              precision    recall  f1-score   support

           0       0.54      0.44      0.49       736
           1       0.24      0.15      0.18       469
           2       0.27      0.20      0.23       630
           3       0.30      0.22      0.25      1207
           4       0.77      0.88      0.82      5460

    accuracy                           0.66      8502
   macro avg       0.42      0.38      0.39      8502
weighted avg       0.61      0.66      0.63      8502

---------------------------------------------
Bigram
---------------------------------------------
              precision    recall  f1-score   support

           0       0.61      0.10      0.17       736
           1       0.11      0.00      0.00       469
           2       0.15      0.00      0.01       630
           3       0.46      0.06      0.11      1207
           4       0.66      0.99      0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


---------------------------------------------
Trigram
---------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.00      0.01       736
           1       1.00      0.00      0.00       469
           2       1.00      0.00      0.00       630
           3       0.60      0.00      0.00      1207
           4       0.64      1.00      0.78      5460

    accuracy                           0.64      8502
   macro avg       0.85      0.20      0.16      8502
weighted avg       0.71      0.64      0.50      8502

---------------------------------------------
Combined N-Grams
---------------------------------------------
              precision    recall  f1-score   support

           0       0.57      0.37      0.45       736
           1       0.28      0.07      0.11       469
           2       0.31      0.12      0.17       630
           3       0.39      0.16      0.23      1207
           4       0.73      0.

...

---------------------------------------------
Combined N-Grams
---------------------------------------------
              precision    recall  f1-score   support

           0       0.74      0.04      0.07       736
           1       1.00      0.00      0.00       469
           2       1.00      0.00      0.00       630
           3       0.00      0.00      0.00      1207
           4       0.65      1.00      0.78      5460

    accuracy                           0.65      8502
   macro avg       0.68      0.21      0.17      8502
weighted avg       0.61      0.65      0.51      8502

Naïve Bayes
---------------------------------------------
Unigram
---------------------------------------------
              precision    recall  f1-score   support

           0       0.65      0.16      0.26       736
           1       0.25      0.00      0.00       469
           2       0.26      0.01      0.03       630
           3       0.29      0.07      0.11      1207
           4       

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


---------------------------------------------
Combined N-Grams
---------------------------------------------
              precision    recall  f1-score   support

           0       1.00      0.00      0.01       736
           1       1.00      0.00      0.00       469
           2       1.00      0.00      0.00       630
           3       0.57      0.00      0.01      1207
           4       0.64      1.00      0.78      5460

    accuracy                           0.64      8502
   macro avg       0.84      0.20      0.16      8502
weighted avg       0.71      0.64      0.50      8502

Gradient Boosting
---------------------------------------------
Unigram
---------------------------------------------
              precision    recall  f1-score   support

           0       0.57      0.17      0.26       736
           1       0.22      0.02      0.04       469
           2       0.22      0.02      0.04       630
           3       0.36      0.07      0.12      1207
           4 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


---------------------------------------------
Trigram
---------------------------------------------
              precision    recall  f1-score   support

           0       0.52      0.02      0.04       736
           1       0.00      0.00      0.00       469
           2       1.00      0.00      0.00       630
           3       0.71      0.02      0.03      1207
           4       0.65      1.00      0.78      5460

    accuracy                           0.65      8502
   macro avg       0.58      0.21      0.17      8502
weighted avg       0.64      0.65      0.51      8502

---------------------------------------------
Combined N-Grams
---------------------------------------------
              precision    recall  f1-score   support

           0       0.58      0.15      0.24       736
           1       0.38      0.03      0.06       469
           2       0.29      0.03      0.05       630
           3       0.43      0.07      0.12      1207
           4       0.67      0.

In [ ]:
texts = train_df['Text'].tolist()
labels = train_df['Score'].tolist()

# Hyperparameters
bert_model_name = 'distilbert-base-uncased'
num_classes = 5
max_len = 128
batch_size = 32
epochs = 5
learning_rate = 2e-5

tokenizer = DistilBertTokenizer.from_pretrained(bert_model_name)
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Prepare raw samples (lazy style)
train_data = list(zip(train_texts, train_labels))
val_data = list(zip(val_texts, val_labels))

# DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_data, batch_size=batch_size, collate_fn=collate_batch)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = create_model(bert_model_name, num_classes).to(device)

# Optional: Use DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    start = time.time()
    avg_loss = train_epoch(model, train_loader, optimizer, scheduler, device)
    print(f"Average Training Loss: {avg_loss:.4f} | Time: {time.time() - start:.2f}s")

    acc, report = evaluate_model(model, val_loader, device)
    print(f"Validation Accuracy: {acc:.4f}")
    print(report)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]


Epoch 1/5


Average Training Loss: 0.6535 | Time: 2866.92s
Validation Accuracy: 0.7678
              precision    recall  f1-score   support

           0       0.59      0.86      0.70      5644
           1       0.46      0.14      0.21      3214
           2       0.50      0.40      0.44      4679
           3       0.58      0.27      0.37      8688
           4       0.85      0.96      0.90     39602

    accuracy                           0.77     61827
   macro avg       0.59      0.52      0.52     61827
weighted avg       0.74      0.77      0.74     61827


Epoch 2/5


KeyboardInterrupt: 